In [1]:
from google.colab import drive
drive.mount("/mnt/drive")

Mounted at /mnt/drive


In [1]:
!pip install -U gdown
import gdown

In [2]:
gdown.download(f"https://drive.google.com/uc?id=1zbD7J9dkiBz3E5eZ4hbydb559jDP7bvN", output="posformer_best_v4.pth", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1zbD7J9dkiBz3E5eZ4hbydb559jDP7bvN
From (redirected): https://drive.google.com/uc?id=1zbD7J9dkiBz3E5eZ4hbydb559jDP7bvN&confirm=t&uuid=d5ce1687-d919-4fa1-9d6e-48b16b8302f3
To: /kaggle/working/posformer_best_v4.pth
100%|██████████| 89.0M/89.0M [00:00<00:00, 101MB/s] 


'posformer_best_v4.pth'

In [4]:
import kagglehub
namphanvanhai_crohme_path = kagglehub.dataset_download('namphanvanhai/crohme')

print('Data source import complete.')


100%|██████████| 25.4M/25.4M [00:00<00:00, 113MB/s]

Extracting files...


Data source import complete.


In [10]:
import os
from os import path
print(os.listdir(path.join(namphanvanhai_crohme_path,"CROHME","2014","img")))

['29_em_153.bmp', '514_em_347.bmp', 'RIT_2014_215.bmp', '512_em_299.bmp', 'RIT_2014_10.bmp', '501_em_21.bmp', 'RIT_2014_39.bmp', 'RIT_2014_199.bmp', '501_em_3.bmp', '34_em_249.bmp', '27_em_118.bmp', '513_em_317.bmp', '506_em_59.bmp', '515_em_372.bmp', '516_em_398.bmp', '516_em_385.bmp', '519_em_440.bmp', 'RIT_2014_20.bmp', '504_em_45.bmp', '36_em_29.bmp', '37_em_31.bmp', '503_em_32.bmp', '506_em_58.bmp', '515_em_351.bmp', 'RIT_2014_229.bmp', '511_em_256.bmp', '501_em_19.bmp', '20_em_43.bmp', 'RIT_2014_161.bmp', '31_em_194.bmp', '29_em_152.bmp', 'RIT_2014_3.bmp', 'RIT_2014_23.bmp', '501_em_17.bmp', '518_em_421.bmp', '502_em_1.bmp', 'RIT_2014_64.bmp', '507_em_75.bmp', '503_em_30.bmp', '31_em_198.bmp', 'RIT_2014_239.bmp', 'RIT_2014_81.bmp', 'RIT_2014_6.bmp', '27_em_102.bmp', '26_em_96.bmp', '32_em_220b.bmp', '26_em_91.bmp', 'RIT_2014_175.bmp', 'RIT_2014_43.bmp', 'RIT_2014_225.bmp', '26_em_78.bmp', '516_em_383.bmp', '36_em_45.bmp', '28_em_125.bmp', '511_em_274.bmp', 'RIT_2014_298.bmp', '51

In [ ]:
!pip install trim
!pip install einops

  Preparing metadata (setup.py) ... done
  Created wheel for trim: filename=trim-0.3-py3-none-any.whl size=3459 sha256=c1a37ad36731c94a4762f97a7e3a4fd8edc7873653de49b90c446b1765fc14b2
  Stored in directory: /root/.cache/pip/wheels/0f/28/33/f5f052e542f8859d0a1b3b897936cb9be800ba768885884b14
Successfully built trim


In [ ]:
# =================================================================================
#               Импорты и базовые библиотеки
# =================================================================================
import os
import re
import math
import random
from collections import defaultdict
from einops import rearrange # Библиотека для удобных операций с тензорами
import numpy as np
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm # Библиотека для создания красивых progress bar'ов
import torchvision.transforms.functional as TF

# =================================================================================
#                                1. ГИПЕРПАРАМЕТРЫ И КОНСТАНТЫ
# =================================================================================
# --- Параметры обработки данных ---
MAX_SEQ_LEN = 512          # Максимальная длина последовательности токенов LaTeX
NUM_NESTED_LEVELS = 10    # Максимальная глубина вложенности структур (a^{b^{c...}})
MAX_IDENTIFIER_LEN = 15   # Максимальная длина строки-идентификатора позиции (e.g., 'MRL' -> 3)
PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN = '<PAD>', '<SOS>', '<EOS>', '<UNK>' # Специальные токены

# --- Параметры архитектуры модели ---
D_MODEL = 256             # Основная размерность векторов в модели (embedding, attention)
D_FF = 1024               # Размерность в скрытом слое Feed-Forward сетей
NUM_HEADS = 8             # Количество голов в Multi-Head Attention
GROWTH_RATE = 24          # "Скорость роста" в DenseNet, количество новых каналов на каждом слое
NUM_ENCODER_LAYERS = 16   # Количество плотных блоков (dense blocks) в энкодере DenseNet
NUM_DECODER_LAYERS = 3    # Количество слоев в декодере
BOTTLENECK = True         # Использовать ли "бутылочное горлышко" (1x1 свертки) в DenseNet для эффективности
DROPOUT_RATE = 0.3        # Вероятность dropout для регуляризации модели

# =================================================================================
#                                2. ТОКЕНИЗАЦИЯ И СЛОВАРИ
# =================================================================================

# Регулярное выражение для грамотного разбиения строки LaTeX на осмысленные токены.
# Оно выделяет:
# 1. Команды LaTeX (e.g., \frac, \sqrt)
# 2. Экранированные скобки и символы
# 3. Числа (включая экспоненциальную запись)
# 4. Буквенные последовательности
# 5. Одиночные математические операторы
# 6. Скобки и знаки препинания
# 7. Пробелы (которые потом игнорируются)
# 8. Любой другой одиночный символ
TOKEN_REGEX = re.compile(
    r"(\\[a-zA-Z]+(?:\*)?)|(\\\{|\\\}|\\.)|([0-9]+(?:\.[0-9]+)?(?:[eE][+-]?[0-9]+)?)|([A-Za-z]+)|([+\-*/=<>!~^_&|%])|([{}()\[\],.;:?'])|(\s+)|(\S)"
)

def tokenize_latex(s: str) -> list[str]:
    """Разбивает строку LaTeX на список токенов с помощью TOKEN_REGEX."""
    tokens_grouped = TOKEN_REGEX.findall(s)
    tokens = []
    for group in tokens_grouped:
        # Из каждой найденной группы берем первый непустой элемент
        non_empty_token = next(filter(None, group), '')
        # Игнорируем пробельные токены
        if not non_empty_token.isspace():
             tokens.append(non_empty_token)
    return tokens

class Vocab:
    """
    Класс для управления словарем токенов. Отвечает за преобразование
    списка токенов в список их числовых ID и обратно.
    """
    def __init__(self, expressions=None, min_freq=1):
        # Инициализация со специальными токенами, которые должны быть в начале словаря
        self.itos = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]
        if expressions:
            # Подсчет частоты каждого токена по всему датасету
            freq = defaultdict(int)
            for expr in expressions:
                for t in tokenize_latex(expr):
                    freq[t] += 1
            # Добавляем в словарь только те токены, которые встречаются чаще `min_freq`
            self.itos.extend([tok for tok, count in sorted(freq.items(), key=lambda x: -x[1]) if count >= min_freq])
        
        # Создаем два словаря-отображения: "токен -> ID" (stoi) и "ID -> токен" (itos)
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}
        self.pad_id = self.stoi[PAD_TOKEN]
        self.sos_id = self.stoi[SOS_TOKEN]
        self.eos_id = self.stoi[EOS_TOKEN]
        self.unk_id = self.stoi[UNK_TOKEN]
        
    def encode(self, tokens: list[str]) -> list[int]:
        """Преобразует список токенов в список их ID."""
        return [self.stoi.get(t, self.unk_id) for t in tokens]

    def decode(self, ids: list[int]) -> str:
        """Преобразует список ID обратно в читаемую строку LaTeX."""
        # Игнорируем специальные токены при декодировании, чтобы получить чистую строку
        tokens = [self.itos[i] for i in ids if i not in {self.pad_id, self.sos_id, self.eos_id}]
        return "".join(tokens)

class PosVocab:
    """Специальный маленький словарь для кодирования позиционных символов ('M', 'L', 'R')."""
    def __init__(self):
        self.itos = ['<PAD>', '<SOS>', '<EOS>', 'M', 'L', 'R']
        self.stoi = {s: i for i, s in enumerate(self.itos)}
        self.pad_id = self.stoi['<PAD>']
        self.sos_id = self.stoi['<SOS>']

# =================================================================================
#                  3. ПАРСЕР ПОЗИЦИЙ
# =================================================================================
class PositionForestEncoder:
    """
    Класс, отвечающий за преобразование строки LaTeX в набор позиционных идентификаторов.
        
    Ее ключевая особенность и фундаментальное отличие от логики в статье PosFormer заключается
    в обработке структурных скобок `{}`. Данный парсер присваивает позиционные метки ('L', 'R')
    только содержимому (контенту) внутри скобок, но не самим скобкам.
    
    Пример: для `\frac{x}{y}` позиционные идентификаторы будут:
    ['M', 'M', 'ML', 'M', 'M', 'MR', 'M']
    Вместо правильного (по статье):
    ['M', 'ML', 'ML', 'ML', 'MR', 'MR', 'MR']
    
    Это упрощение задачи для модели на ранних этапах, которое позволило быстро дойти до 43%
    """
    def __init__(self, pos_vocab, max_len=MAX_SEQ_LEN, max_identifier_len=MAX_IDENTIFIER_LEN):
        self.pos_vocab = pos_vocab
        self.max_len = max_len
        self.max_identifier_len = max_identifier_len
        self._cache = {} # Кэширование для ускорения работы с повторяющимися формулами

    def _get_pos_strings(self, tokens):
        """
        Основная логика парсинга. Рекурсивно обходит токены и строит строки позиций.
        """
        cache_key = tuple(tokens)
        if cache_key in self._cache:
            return self._cache[cache_key]

        T = len(tokens)
        # Инициализируем все токены базовой позицией 'M' (middle/main)
        ids = ['M'] * T

        def find_matching_brace(start_index):
            """Вспомогательная функция для поиска парной закрывающей скобки, учитывая вложенность."""
            depth = 1
            for i in range(start_index + 1, T):
                if tokens[i] == '{':
                    depth += 1
                elif tokens[i] == '}':
                    depth -= 1
                    if depth == 0:
                        return i
            return T - 1 # Возвращаем последний индекс, если пара не найдена

        def parse_recursive(start, end, current_pos_prefix):
            """
            Рекурсивная функция, обходящая последовательность токенов.
            `current_pos_prefix` хранит текущий контекст вложенности (e.g., 'M', 'ML', 'MLR').
            """
            i = start
            while i < end:
                tok = tokens[i]

                # Обработка команд с одним аргументом: ^ (верхний индекс), _ (нижний), \sqrt (корень)
                if tok in ('^', '_', '\\sqrt') and i + 1 < end:
                    # 'L' (Left/upper) для верхних индексов и корней, 'R' (Right/lower) для нижних
                    label = 'L' if tok in ('^', '\\sqrt') else 'R'
                    
                    if tokens[i+1] == '{': # Если аргумент в скобках
                        brace_end = find_matching_brace(i + 1)
                        
                        # !!! КЛЮЧЕВАЯ ОСОБЕННОСТЬ ЭТОГО ПАРСЕРА !!!
                        # Цикл идет от i+2 (индекс токена ПОСЛЕ '{') до brace_end (индекс токена ПЕРЕД '}').
                        # Таким образом, сами скобки `{` и `}` НЕ ПОЛУЧАЮТ новую позиционную метку.
                        # Они остаются с их исходной меткой 'M'.
                        for k in range(i + 2, brace_end):
                            ids[k] = current_pos_prefix + label
                        
                        # Рекурсивный вызов для разбора содержимого скобок
                        parse_recursive(i + 2, brace_end, current_pos_prefix + label)
                        i = brace_end # Перепрыгиваем через весь обработанный блок
                    else: # Если аргумент - это один токен без скобок
                        ids[i+1] = current_pos_prefix + label
                        i += 1 # Пропускаем команду и ее аргумент

                # Обработка команды \frac с двумя аргументами
                elif tok == '\\frac' and i + 1 < end and tokens[i+1] == '{':
                    num_end = find_matching_brace(i + 1) # Находим конец числителя
                    if num_end + 1 < end and tokens[num_end + 1] == '{':
                        den_end = find_matching_brace(num_end + 1) # Находим конец знаменателя
                        
                        # --- Числитель (Numerator) ---
                        # !!! СНОВА ТА ЖЕ ЛОГИКА: позиционная метка 'L' применяется только к контенту внутри скобок.
                        for k in range(i + 2, num_end):
                            ids[k] = current_pos_prefix + 'L'
                        parse_recursive(i + 2, num_end, current_pos_prefix + 'L')
                        
                        # --- Знаменатель (Denominator) ---
                        # Позиционная метка 'R' также применяется только к контенту.
                        for k in range(num_end + 2, den_end):
                            ids[k] = current_pos_prefix + 'R'
                        parse_recursive(num_end + 2, den_end, current_pos_prefix + 'R')
                        
                        i = den_end # Перепрыгиваем через всю дробь
                    else:
                        # Если у \frac только один аргумент (синтаксически неверно, но обрабатываем), пропускаем его
                        i = num_end
                
                i += 1
        
        # Запускаем рекурсивный парсинг для всей последовательности токенов
        parse_recursive(0, T, 'M')
        
        # Пост-обработка не требуется, так как ids уже содержит полные строки, но для консистентности
        # создается новый список.
        final_ids = []
        for i in range(T):
            if ids[i] == 'M':
                final_ids.append('M')
            else:
                final_ids.append(ids[i])

        self._cache[cache_key] = final_ids
        return final_ids

    def process_formula(self, latex_str: str):
        """
        Главный метод класса. Превращает одну строку LaTeX в словарь с тензорами для обучения.
        """
        # 1. Токенизация и обрезка по максимальной длине
        tokens = tokenize_latex(latex_str)
        tokens = tokens[:self.max_len - 2]
        
        # 2. Получение строк позиций от нашего парсера
        pos_strings_raw = self._get_pos_strings(tokens)
        
        # 3. Формирование ground-truth для вспомогательных задач
        #    - Уровень вложенности (nested_depth): длина строки 'MLR' -> 3, уровень -> 2
        #    - Относительная позиция (rel_pos): последний символ строки ('M', 'L' или 'R')
        tokens_gt = [SOS_TOKEN] + tokens + [EOS_TOKEN]
        nested_depth_gt = [0] + [min(len(pid) - 1, NUM_NESTED_LEVELS) for pid in pos_strings_raw] + [0]
        rel_pos_gt = [0] + [1 if pid.endswith('L') else 2 if pid.endswith('R') else 0 for pid in pos_strings_raw] + [0]
        
        # 4. Кодирование строк позиций в числовые ID и создание позиционной матрицы
        pos_identifiers_ids = []
        # Позиция для SOS токена (стандартная)
        sos_pos_ids = [self.pos_vocab.sos_id] + [self.pos_vocab.stoi['M']] + [self.pos_vocab.stoi['<EOS>']]
        pos_identifiers_ids.append(sos_pos_ids)
        # Позиции для каждого токена формулы
        for pos_str in pos_strings_raw:
            ids = [self.pos_vocab.stoi.get(c, 0) for c in pos_str]
            ids = [self.pos_vocab.sos_id] + ids + [self.pos_vocab.stoi['<EOS>']]
            pos_identifiers_ids.append(ids)
        # Позиция для EOS токена
        pos_identifiers_ids.append(sos_pos_ids)
        
        # 5. Паддинг всех последовательностей до единой длины
        final_len = len(tokens_gt)
        tokens_gt_padded = tokens_gt + [PAD_TOKEN] * (self.max_len - final_len)
        nested_depth_gt_padded = nested_depth_gt + [0] * (self.max_len - final_len)
        rel_pos_gt_padded = rel_pos_gt + [0] * (self.max_len - final_len)
        
        # Паддинг каждой строки в позиционной матрице до MAX_IDENTIFIER_LEN
        for i in range(len(pos_identifiers_ids)):
            seq = pos_identifiers_ids[i][:self.max_identifier_len]
            pos_identifiers_ids[i] = seq + [self.pos_vocab.pad_id] * (self.max_identifier_len - len(seq))
        
        # Паддинг самой матрицы (добавление пустых строк для паддинга токенов)
        empty_pos_id_seq = [self.pos_vocab.pad_id] * self.max_identifier_len
        padded_pos_ids = pos_identifiers_ids + [empty_pos_id_seq] * (self.max_len - final_len)
        
        # Возвращаем словарь с готовыми к отправке в модель тензорами
        return {
            "tokens_gt": tokens_gt_padded,
            "pos_matrix": torch.tensor(padded_pos_ids, dtype=torch.long),
            "nested_gt": torch.tensor(nested_depth_gt_padded, dtype=torch.long),
            "rel_pos_gt": torch.tensor(rel_pos_gt_padded, dtype=torch.long)
        }
# =================================================================================
#               3. КЛАССЫ ДЛЯ ПРЕ-ОБРАБОТКИ ИЗОБРАЖЕНИЙ И DATASET
# =================================================================================

# В этой версии кода используется более гибкий подход к аугментации и изменению размера.
# Вместо одного `transforms.Compose` в `__init__`, трансформации применяются
# вручную и последовательно в методе `__getitem__`. Это дает больше контроля.

class ResizeWithPadding:
    """
    Класс для изменения размера изображения до целевой высоты с сохранением пропорций
    и последующим добавлением паддинга до максимальной ширины.
    (Этот класс присутствует в коде, но в `CROHMEDataset` не используется,
     вместо него применяется более гибкая связка ScaleToLimitRange + ручной паддинг в collate_fn)
    """
    def __init__(self, target_height, max_target_width, padding_value=255):
        self.target_height = target_height
        self.max_target_width = max_target_width
        self.padding_value = padding_value
        
    def __call__(self, img):
        w, h = img.size
        # Вычисляем новую ширину, сохраняя пропорции
        new_w = int(w * (self.target_height / h))
        # Ограничиваем максимальную ширину
        new_w = min(new_w, self.max_target_width)
        img_resized = img.resize((new_w, self.target_height), Image.LANCZOS)
        
        # Создаем новое "пустое" изображение (холст) с целевыми размерами
        new_img = Image.new(img.mode, (self.max_target_width, self.target_height), self.padding_value)
        # Вставляем измененное изображение в левый верхний угол холста
        new_img.paste(img_resized, (0, 0))
        
        # Создаем маску для паддинга, чтобы модель знала, где реальное изображение, а где пустота
        mask = torch.ones((self.target_height, self.max_target_width), dtype=torch.bool)
        mask[:, :new_w] = False # False - реальные пиксели, True - паддинг
        return new_img, mask

class ScaleToLimitRange:
    """
    Трансформация, которая пропорционально изменяет размер изображения так,
    чтобы его высота и ширина попали в заданные диапазоны [h_lo, h_hi] и [w_lo, w_hi].
    Работает с numpy-массивами.
    """
    def __init__(self, w_lo: int, w_hi: int, h_lo: int, h_hi: int) -> None:
        assert w_lo <= w_hi and h_lo <= h_hi
        self.w_lo = w_lo
        self.w_hi = w_hi
        self.h_lo = h_lo
        self.h_hi = h_hi

    def __call__(self, img: np.ndarray) -> np.ndarray:
        h, w = img.shape[:2]
        
        # Если изображение слишком большое хотя бы по одному измерению, сжимаем его
        scale_r = min(self.h_hi / h, self.w_hi / w)
        if scale_r < 1.0:
            img = cv2.resize(img, None, fx=scale_r, fy=scale_r, interpolation=cv2.INTER_LINEAR)
            return img

        # Если изображение слишком маленькое хотя бы по одному измерению, растягиваем его
        scale_r = max(self.h_lo / h, self.w_lo / w)
        if scale_r > 1.0:
            img = cv2.resize(img, None, fx=scale_r, fy=scale_r, interpolation=cv2.INTER_LINEAR)
            return img

        # Если размеры уже в пределах нормы, ничего не делаем
        return img
        
class ScaleAugmentation:
    """
    Трансформация для аугментации данных: случайное пропорциональное
    изменение масштаба изображения в заданном диапазоне [lo, hi].
    Применяется только на этапе обучения.
    """
    def __init__(self, lo: float, hi: float) -> None:
        assert lo <= hi
        self.lo = lo
        self.hi = hi

    def __call__(self, img: np.ndarray) -> np.ndarray:
        # Выбираем случайный коэффициент масштабирования
        k = np.random.uniform(self.lo, self.hi)
        img = cv2.resize(img, None, fx=k, fy=k, interpolation=cv2.INTER_LINEAR)
        return img

class CROHMEDataset(Dataset):
    """
    Основной класс датасета. Загружает изображения и соответствующие им LaTeX-формулы,
    применяет трансформации и подготавливает данные для модели.
    """
    def __init__(self, base_dir, caption_file, vocab, pos_vocab, is_train=True):
        self.vocab = vocab
        self.pfe = PositionForestEncoder(pos_vocab) # Инициализируем наш парсер позиций
        self.items = []
        self.is_train = is_train

        # Формируем список пар (путь_к_изображению, строка_latex)
        img_dir = os.path.join(base_dir, 'img')
        formulas_path = os.path.join(base_dir, caption_file)
        with open(formulas_path, 'r', encoding='utf-8') as f:
            for line in f:
                fid, latex = line.strip().split('\t')
                self.items.append((os.path.join(img_dir, f"{fid}.bmp"), latex))

        # Определяем диапазоны размеров и инициализируем классы трансформаций
        H_MIN, H_MAX = 32, 256
        W_MIN, W_MAX = 32, 512
        
        self.scale_augmenter = None
        if self.is_train:
            # Аугментация масштаба применяется только для обучающей выборки
            self.scale_augmenter = ScaleAugmentation(0.8, 1.2)
        
        # Контроллер размера применяется всегда (и для train, и для val)
        self.size_controller = ScaleToLimitRange(h_lo=H_MIN, h_hi=H_MAX, w_lo=W_MIN, w_hi=W_MAX)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, latex = self.items[idx]
        try:
            # Загружаем изображение с помощью OpenCV в виде numpy-массива в оттенках серого
            img_np = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img_np is None: 
                raise FileNotFoundError(f"Не удалось прочитать файл: {path}")
        except Exception:
            # В случае ошибки (битый файл, и т.д.) рекурсивно берем следующий элемент датасета
            return self.__getitem__((idx + 1) % len(self))
            
        # --- Ручной пайплайн применения трансформаций ---
        # Этот подход дает больше гибкости, чем стандартный `transforms.Compose`.

        # Шаг 1: Аугментация масштаба (только для train)
        if self.scale_augmenter:
            img_np = self.scale_augmenter(img_np)
        
        # Шаг 2: Приведение к целевому диапазону размеров
        img_np = self.size_controller(img_np)

        # Шаг 3: Конвертация в PIL.Image для дальнейшей обработки в DataLoader'е
        final_img_pil = Image.fromarray(img_np)
        
        # --- Обработка LaTeX-формулы ---
        
        # Получаем словарь с тензорами позиций от нашего парсера
        data = self.pfe.process_formula(latex)
        # Кодируем ground-truth токены в их ID
        token_ids = torch.tensor(self.vocab.encode(data["tokens_gt"]), dtype=torch.long)
        
        # Возвращаем все необходимые данные. collate_fn позже соберет их в батч.
        return (final_img_pil, token_ids, data["pos_matrix"], data["nested_gt"], data["rel_pos_gt"], latex)

# =================================================================================
#                         4. ЭНКОДЕР (КАСТОМНАЯ РЕАЛИЗАЦИЯ DenseNet)
# =================================================================================

class ImgPosEnc(nn.Module):
    """
    Создает 2D позиционные кодировки для фичер-мапы изображения, аналогично
    позиционным кодировкам в трансформерах, но для двух измерений (высота и ширина).
    Это позволяет модели понимать пространственное расположение признаков.
    """
    def __init__(self, d_model: int, temperature: float = 10000.0, normalize: bool = True, scale: float = None):
        super().__init__()
        if d_model % 4 != 0: raise ValueError(f"d_model ({d_model}) должен быть кратен 4.")
        self.d_model = d_model
        self.temperature = temperature
        self.normalize = normalize
        if scale is None: scale = 2 * math.pi
        self.scale = scale

    def forward(self, x: torch.Tensor, mask: torch.BoolTensor) -> torch.Tensor:
        # `mask` здесь - это маска паддинга (False - где есть пиксели, True - где паддинг)
        not_mask = ~mask
        # Создаем координатные сетки: y_embed - вертикальные, x_embed - горизонтальные
        y_embed = not_mask.cumsum(1, dtype=torch.float32)
        x_embed = not_mask.cumsum(2, dtype=torch.float32)
        
        if self.normalize:
            # Нормализуем координаты в диапазон [0, scale] для стабильности
            eps = 1e-6
            y_embed = (y_embed / (y_embed[:, -1:, :] + eps)) * self.scale
            x_embed = (x_embed / (x_embed[:, :, -1:] + eps)) * self.scale

        # Формула позиционного кодирования (sin/cos на разных частотах)
        dim_t_half = self.d_model // 2
        dim_t = torch.arange(dim_t_half, dtype=torch.float32, device=x.device)
        dim_t = self.temperature ** (2 * (dim_t // 2) / dim_t_half)
        
        pos_x = x_embed[:, :, :, None] / dim_t
        pos_y = y_embed[:, :, :, None] / dim_t
        
        # Чередуем sin и cos
        pos_x = torch.stack((pos_x[:, :, :, 0::2].sin(), pos_x[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos_y = torch.stack((pos_y[:, :, :, 0::2].sin(), pos_y[:, :, :, 1::2].cos()), dim=4).flatten(3)
        
        # Конкатенируем кодировки по X и Y, и прибавляем к входному тензору
        pos = torch.cat((pos_y, pos_x), dim=3)
        return x + pos

class _Bottleneck(nn.Module):
    """Внутренний блок "бутылочное горлышко" для DenseNet."""
    def __init__(self, n_channels: int, growth_rate: int, use_dropout: bool):
        super(_Bottleneck, self).__init__()
        interChannels = 4 * growth_rate
        # Сначала 1x1 свертка для уменьшения каналов (узкое место)
        self.conv1 = nn.Conv2d(n_channels, interChannels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(interChannels)
        # Затем основная 3x3 свертка
        self.conv2 = nn.Conv2d(interChannels, growth_rate, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(growth_rate)
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        if self.use_dropout: out = self.dropout(out)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        if self.use_dropout: out = self.dropout(out)
        # Ключевая идея DenseNet: конкатенация входа и выхода по канальному измерению
        out = torch.cat((x, out), 1)
        return out

class _Transition(nn.Module):
    """Переходный слой в DenseNet. Уменьшает количество каналов и пространственный размер."""
    def __init__(self, n_channels: int, n_out_channels: int, use_dropout: bool):
        super(_Transition, self).__init__()
        self.conv1 = nn.Conv2d(n_channels, n_out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_out_channels)
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        if self.use_dropout: out = self.dropout(out)
        # Уменьшение пространственного размера (downsampling)
        out = F.avg_pool2d(out, 2, ceil_mode=True)
        return out

class DenseNet(nn.Module):
    """Кастомная реализация архитектуры DenseNet, адаптированная для задачи."""
    def __init__(self, growth_rate: int, num_layers: int, reduction: float = 0.5, bottleneck: bool = True, use_dropout: bool = True):
        super(DenseNet, self).__init__()
        n_dense_blocks = num_layers
        n_channels = 2 * growth_rate
        
        # Первый сверточный слой
        self.conv1 = nn.Conv2d(1, n_channels, kernel_size=7, padding=3, stride=2, bias=False) # 1 канал на входе (grayscale)
        self.norm1 = nn.BatchNorm2d(n_channels)
        
        # Первый блок плотных слоев
        self.dense1 = self._make_dense(n_channels, growth_rate, n_dense_blocks, bottleneck, use_dropout)
        n_channels += n_dense_blocks * growth_rate
        n_out_channels = int(math.floor(n_channels * reduction))
        self.trans1 = _Transition(n_channels, n_out_channels, use_dropout)
        
        # Второй блок плотных слоев
        n_channels = n_out_channels
        self.dense2 = self._make_dense(n_channels, growth_rate, n_dense_blocks, bottleneck, use_dropout)
        n_channels += n_dense_blocks * growth_rate
        n_out_channels = int(math.floor(n_channels * reduction))
        self.trans2 = _Transition(n_channels, n_out_channels, use_dropout)
        
        # Третий блок плотных слоев
        n_channels = n_out_channels
        self.dense3 = self._make_dense(n_channels, growth_rate, n_dense_blocks, bottleneck, use_dropout)
        self.out_channels = n_channels + n_dense_blocks * growth_rate
        self.post_norm = nn.BatchNorm2d(self.out_channels)

    @staticmethod
    def _make_dense(n_channels, growth_rate, n_dense_blocks, bottleneck, use_dropout):
        """Собирает последовательность из нескольких плотных блоков."""
        layers = []
        for _ in range(int(n_dense_blocks)):
            if bottleneck:
                layers.append(_Bottleneck(n_channels, growth_rate, use_dropout))
            n_channels += growth_rate
        return nn.Sequential(*layers)
        
    def forward(self, x, x_mask):
        # Прогоняем изображение через всю сеть, корректно изменяя маску на каждом шаге downsampling'а
        out = self.conv1(x)
        out = self.norm1(out)
        out_mask = x_mask[:, ::2, ::2]
        
        out = F.relu(out, inplace=True)
        out = F.max_pool2d(out, 2, ceil_mode=True)
        out_mask = out_mask[:, ::2, ::2]
        
        out = self.dense1(out)
        out = self.trans1(out)
        out_mask = out_mask[:, ::2, ::2]
        
        out = self.dense2(out)
        out = self.trans2(out)
        out_mask = out_mask[:, ::2, ::2]
        
        out = self.dense3(out)
        out = self.post_norm(out)
        return out, out_mask

class Encoder(nn.Module):
    """
    Основной класс энкодера, который объединяет DenseNet для извлечения признаков
    и 2D позиционное кодирование.
    """
    def __init__(self, d_model, growth_rate, num_layers, dropout):
        super().__init__()
        self.densenet = DenseNet(growth_rate, num_layers)
        # Проекция из количества каналов DenseNet в рабочую размерность модели d_model
        self.feature_proj = nn.Conv2d(self.densenet.out_channels, d_model, 1)
        self.pos_enc_2d = ImgPosEnc(d_model, normalize=True)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, img, img_mask):
        # 1. Извлекаем фичер-мапу из DenseNet
        feature, mask = self.densenet(img, img_mask)
        # 2. Проецируем в нужное кол-во каналов
        feature = self.feature_proj(feature)
        
        # 3. Добавляем 2D позиционные кодировки
        feature_permuted = feature.permute(0, 2, 3, 1) # Переставляем размерности для LayerNorm
        pos_encoded_feature = self.pos_enc_2d(feature_permuted, mask)
        
        # 4. Нормализация и dropout
        normed_feature = self.norm(pos_encoded_feature)
        dropped_feature = self.dropout(normed_feature)

        # 5. "Выравниваем" 2D фичер-мапу в 1D последовательность для декодера-трансформера
        return rearrange(dropped_feature, "b h w d -> b (h w) d"), rearrange(mask, "b h w -> b (h w)")

# =================================================================================
#                     5. ДЕКОДЕР И МОДУЛЬ КОРРЕКЦИИ ВНИМАНИЯ (IAC)
# =================================================================================

class StandardCrossAttention(nn.Module):
    """Обертка для стандартного Multi-Head Attention, чтобы его интерфейс совпадал с IAC."""
    def __init__(self, d_model, num_heads, dropout):
        super().__init__()
        self.mha = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        
    def forward(self, q, k, v, ids, pad_mask): # `ids` здесь не используется, но нужен для совместимости
        return self.mha(q, k, v, key_padding_mask=pad_mask)[0]

class IAC(nn.Module):
    """
    Implicit Attention Correction. Продвинутый механизм внимания, который пытается
    решить проблему "пере-внимания" на уже распознанные участки, особенно для
    структурных символов, у которых нет своего изображения.
    """
    def __init__(self, d_model, num_heads, dropout, vocab):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.wq, self.wk, self.wv, self.wo = [nn.Linear(d_model, d_model, bias=False) for _ in range(4)]
        # Свертка и линейный слой для модуля коррекции phi
        self.phi_conv = nn.Conv2d(num_heads, num_heads, kernel_size=3, padding=1, groups=num_heads)
        self.phi_linear = nn.Linear(self.num_heads, self.head_dim)
        self.dropout = nn.Dropout(dropout)

        # Собираем ID всех "структурных" токенов, которые не имеют прямого аналога на картинке
        structure_symbols = {
            '^', '_', '{', '}', '\\frac', '\\sqrt', '\\left', '\\right',
            '\\big', '\\Big', '\\bigg', '\\Bigg', '&', '\\\\',
            PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN
        }
        s_ids = [vocab.stoi[s] for s in structure_symbols if s in vocab.stoi]
        self.register_buffer('structure_ids', torch.tensor(s_ids, dtype=torch.long))

    def _compute_phi(self, accum_attn):
        """Вспомогательная функция для вычисления корректирующего члена phi."""
        B, H, Lq, Lk = accum_attn.shape
        s_dim_float = math.sqrt(Lk)
        # Эта реализация предполагает, что фичер-мапу можно "свернуть" в квадрат
        if s_dim_float != int(s_dim_float): return 0
        s_dim = int(s_dim_float)

        # Применяем 2D свертку к накопленным весам внимания
        phi_in = rearrange(accum_attn, 'b h lq (s1 s2) -> (b lq) h s1 s2', s1=s_dim, s2=s_dim)
        conv_out = self.phi_conv(phi_in)
        phi_permuted = rearrange(conv_out, '(b lq) h s1 s2 -> b lq (s1 s2) h', b=B)
        phi_features = self.phi_linear(phi_permuted)
        return phi_features.unsqueeze(1)

    def forward(self, q, k, v, ids, pad_mask):
        B, Lq, _ = q.shape; Lk = k.shape[1]
        
        # Стандартные проекции Q, K, V
        Q = self.wq(q).view(B, Lq, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.wk(k).view(B, Lk, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.wv(v).view(B, Lk, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # --- Логика IAC ---
        # 1. Получаем временные веса внимания (без градиента)
        temp_attn_weights = F.softmax(scores, dim=-1).detach()
        # 2. Создаем индикатор, который равен 1 для обычных символов и 0 для структурных
        indicator = (~(ids.unsqueeze(-1) == self.structure_ids).any(-1)).float().view(B, 1, Lq, 1)
        # 3. "Обнуляем" внимание от структурных символов
        masked_attn = temp_attn_weights * indicator
        # 4. Накапливаем "правильное" внимание с предыдущих шагов
        shifted = torch.zeros_like(masked_attn)
        if Lq > 1: shifted[:, :, 1:, :] = masked_attn[:, :, :-1, :]
        accumulated_attention = torch.cumsum(shifted, dim=2)

        # 5. Вычисляем и вычитаем корректирующий член
        phi = self._compute_phi(accumulated_attention)
        correction = (Q.unsqueeze(3) * phi).sum(dim=-1) if isinstance(phi, torch.Tensor) else 0
        corrected_scores = scores - correction

        # Стандартное завершение
        if pad_mask is not None:
            corrected_scores = corrected_scores.masked_fill(pad_mask.unsqueeze(1).unsqueeze(2), float('-inf'))
        final_attn_weights = F.softmax(corrected_scores, dim=-1)
        context = torch.matmul(self.dropout(final_attn_weights), V).transpose(1, 2).reshape(B, Lq, self.d_model)
        return self.wo(context)

class EnhancedDecoderLayer(nn.Module):
    """Один слой декодера, который включает в себя self-attention, cross-attention (обычный или IAC) и FFN."""
    def __init__(self, d_model, num_heads, d_ff, dropout, cross_attention_module):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        # Используем переданный модуль, что позволяет гибко менять тип cross-attention
        self.cross_attn = cross_attention_module
        self.ffn = nn.Sequential(nn.Linear(d_model, d_ff), nn.GELU(), nn.Linear(d_ff, d_model))
        self.norm1, self.norm2, self.norm3 = [nn.LayerNorm(d_model) for _ in range(3)]
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, ids, tgt_mask, tgt_pad_mask, mem_pad_mask):
        # 1. Self-attention (внимание на предыдущие сгенерированные токены)
        x = x + self.dropout(self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), attn_mask=tgt_mask, key_padding_mask=tgt_pad_mask)[0])
        # 2. Cross-attention (внимание на выход энкодера)
        x = x + self.dropout(self.cross_attn(self.norm2(x), enc_out, enc_out, ids, mem_pad_mask))
        # 3. Feed-forward network
        x = x + self.dropout(self.ffn(self.norm3(x)))
        return x

# =================================================================================
#                      6. ПОЛНАЯ МОДЕЛЬ POSFORMER И ВСПОМОГАТЕЛЬНЫЕ КЛАССЫ
# =================================================================================
class LabelSmoothingCrossEntropy(nn.Module):
    """
    Реализация Cross-Entropy с "сглаживанием меток" (Label Smoothing).
    Это мощный метод регуляризации, который не дает модели становиться
    излишне самоуверенной в своих предсказаниях, что улучшает обобщение.
    Вместо того чтобы требовать от модели предсказать [0, 0, 1, 0], мы просим ее
    предсказать что-то вроде [0.01, 0.01, 0.95, 0.01].
    """
    def __init__(self, smoothing=0.0):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        
    def forward(self, x, target, ignore_index=-100):
        confidence = 1. - self.smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1)).squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + self.smoothing * smooth_loss
        # Игнорируем потери для паддинг-токенов
        mask = (target != ignore_index)
        return (loss * mask.float()).sum() / mask.float().sum()

class PosFormer(nn.Module):
    """
    Основной класс модели, собирающий все компоненты воедино.
    """
    def __init__(self, main_vocab, pos_vocab):
        super().__init__()
        self.vocab, self.pos_vocab = main_vocab, pos_vocab
        self.pad_id, self.sos_id, self.eos_id = main_vocab.pad_id, main_vocab.sos_id, main_vocab.eos_id
        
        # --- Компоненты модели ---
        self.encoder = Encoder(D_MODEL, GROWTH_RATE, NUM_ENCODER_LAYERS, DROPOUT_RATE)
        # Эмбеддинги для токенов LaTeX
        self.emb_tok = nn.Embedding(len(main_vocab.itos), D_MODEL)
        # Эмбеддинги для позиционных символов ('M', 'L', 'R')
        self.pos_id_emb = nn.Embedding(len(pos_vocab.itos), D_MODEL)
        # Линейная проекция для "сплющенной" матрицы позиций
        self.xi_proj = nn.Sequential(nn.Linear(D_MODEL * MAX_IDENTIFIER_LEN, D_MODEL), nn.GELU(), nn.LayerNorm(D_MODEL))

        # Создаем слои декодера: первый слой - обычный Cross-Attention, остальные - с IAC
        decoder_layers = []
        for i in range(NUM_DECODER_LAYERS):
            use_iac = i > 0 # IAC на 2м и 3м слое
            cross_attn_module = IAC(D_MODEL, NUM_HEADS, DROPOUT_RATE, main_vocab) if use_iac \
                else StandardCrossAttention(D_MODEL, NUM_HEADS, DROPOUT_RATE)
            decoder_layers.append(
                EnhancedDecoderLayer(D_MODEL, NUM_HEADS, D_FF, DROPOUT_RATE, cross_attn_module)
            )
        self.decoders = nn.ModuleList(decoder_layers)

        self.dec_norm = nn.LayerNorm(D_MODEL)
        
        # --- "Головы" для предсказания (три задачи) ---
        self.head_tok = nn.Linear(D_MODEL, len(main_vocab.itos))      # 1. Предсказание следующего токена LaTeX
        self.head_nested = nn.Linear(D_MODEL, NUM_NESTED_LEVELS + 1) # 2. Предсказание уровня вложенности
        self.head_rel_pos = nn.Linear(D_MODEL, 3)                    # 3. Предсказание относительной позиции (M, L, R)

        # Функция потерь с Label Smoothing для основной задачи
        self.loss_fn_rec = LabelSmoothingCrossEntropy(smoothing=0.05)
        self._init_weights()
        self.rel_pos_map = {0: 'M', 1: 'L', 2: 'R'}

    def _init_weights(self):
        """Инициализация весов для лучшей сходимости."""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, imgs, img_mask, pos_matrix, token_ids):
        # 1. Получаем фичи из энкодера
        vis_feats, mem_pad_mask = self.encoder(imgs, img_mask)
        
        # 2. Готовим вход для декодера
        # 2.1. Получаем эмбеддинги для позиций
        pos_input = self.xi_proj(rearrange(self.pos_id_emb(pos_matrix[:, :-1, :]), 'b l d e -> b l (d e)'))
        # 2.2. Получаем эмбеддинги для токенов
        token_input_ids = token_ids[:, :-1]
        token_embeds = self.emb_tok(token_input_ids)
        # 2.3. Суммируем эмбеддинги токенов и позиций
        tgt = pos_input + token_embeds
        
        # 3. Готовим маски для декодера
        # Маска, чтобы декодер не "подсматривал" в будущее
        tgt_mask = torch.triu(torch.ones(tgt.size(1), tgt.size(1), device=tgt.device), 1).bool()
        # Маска для паддинг-токенов в последовательности
        tgt_pad_mask = (token_input_ids == self.pad_id)
        
        # 4. Прогоняем через декодер
        dec_out = self.dec_norm(self._decode(tgt, vis_feats, token_input_ids, tgt_mask, tgt_pad_mask, mem_pad_mask))
        
        # 5. Получаем предсказания из голов
        return self.head_tok(dec_out), self.head_nested(dec_out), self.head_rel_pos(dec_out)

    def _decode(self, tgt, mem, ids, tgt_mask, tgt_pad_mask, mem_pad_mask):
        """Пропускает данные через все слои декодера."""
        for layer in self.decoders:
            tgt = layer(tgt, mem, ids, tgt_mask, tgt_pad_mask, mem_pad_mask)
        return tgt

    def compute_loss(self, logits, targets):
        """Вычисляет общую потерю как сумму потерь по трем задачам."""
        token_logits, nested_logits, rel_pos_logits = logits
        token_ids_gt, nested_gt, rel_pos_gt = targets
        token_targets = token_ids_gt[:, 1:]
        nested_targets = nested_gt[:, 1:]
        rel_pos_targets = rel_pos_gt[:, 1:]

        # Потери для основной задачи распознавания (с Label Smoothing)
        loss_rec = self.loss_fn_rec(
            token_logits.reshape(-1, token_logits.size(-1)),
            token_targets.reshape(-1),
            ignore_index=self.pad_id
        )
        
        # Вычисляем потери для вспомогательных задач только для не-паддинг токенов
        mask = (token_targets != self.pad_id).flatten()
        if not mask.any(): # Если в батче только паддинг
            return {'total': loss_rec, 'rec': loss_rec, 'pos': torch.tensor(0.0, device=token_logits.device)}

        loss_nested = F.cross_entropy(nested_logits.reshape(-1, nested_logits.size(-1))[mask],
                                      nested_targets.reshape(-1)[mask])
        loss_rel = F.cross_entropy(rel_pos_logits.reshape(-1, rel_pos_logits.size(-1))[mask],
                                   rel_pos_targets.reshape(-1)[mask])
        loss_pos = loss_nested + loss_rel
        
        # Общая потеря - взвешенная сумма (здесь веса = 1)
        total_loss = loss_rec + loss_pos

        return {'total': total_loss, 'rec': loss_rec, 'pos': loss_pos}
    def _construct_next_pos_string(self, prev_pos_str: str, pred_nested_level: int, pred_rel_pos: int) -> str:
        prev_level = len(prev_pos_str) - 1
        new_pos_str = prev_pos_str[:pred_nested_level + 1]
        if pred_nested_level > prev_level:
            if len(new_pos_str) == prev_level + 1:
                 new_pos_str += self.rel_pos_map.get(pred_rel_pos, 'M')
        return new_pos_str

    @torch.no_grad()
    def generate(self, imgs, max_gen_len=150):
        self.eval()
        B = imgs.shape[0]; device = imgs.device
        img_mask = torch.zeros_like(imgs[:, 0, :, :], dtype=torch.bool)
        vis_feats, mem_pad_mask = self.encoder(imgs, img_mask)
        generated_ids = torch.full((B, 1), self.sos_id, dtype=torch.long, device=device)
        pos_strings_T = [['M'] for _ in range(B)]
        is_finished = torch.zeros(B, dtype=torch.bool, device=device)
        for t in range(max_gen_len - 1):
            token_embeds = self.emb_tok(generated_ids)
            pos_ids_list = []
            max_len_in_batch = max(len(p_list) for p_list in pos_strings_T)
            for i in range(B):
                batch_pos_ids = []
                for p_str in pos_strings_T[i]:
                    ids = [self.pos_vocab.sos_id] + [self.pos_vocab.stoi.get(c,0) for c in p_str] + [self.pos_vocab.stoi['<EOS>']]
                    padded_ids = ids[:MAX_IDENTIFIER_LEN] + [self.pos_vocab.pad_id] * (MAX_IDENTIFIER_LEN - len(ids))
                    batch_pos_ids.append(torch.tensor(padded_ids, device=device))
                while len(batch_pos_ids) < max_len_in_batch:
                    batch_pos_ids.append(torch.full((MAX_IDENTIFIER_LEN,), self.pos_vocab.pad_id, device=device))
                pos_ids_list.append(torch.stack(batch_pos_ids))
            pos_matrix = torch.stack(pos_ids_list)
            pos_embeds = self.xi_proj(rearrange(self.pos_id_emb(pos_matrix), 'b l d e -> b l (d e)'))
            tgt = token_embeds + pos_embeds
            tgt_mask = torch.triu(torch.ones(tgt.size(1), tgt.size(1), device=device), 1).bool()
            dec_out = self.dec_norm(self._decode(tgt, vis_feats, generated_ids, tgt_mask, None, mem_pad_mask))
            last_step_out = dec_out[:, -1, :]
            token_logits, nested_logits, rel_pos_logits = self.head_tok(last_step_out), self.head_nested(last_step_out), self.head_rel_pos(last_step_out)
            next_token_id = torch.argmax(token_logits, dim=-1).unsqueeze(1)
            pred_nested_level, pred_rel_pos = torch.argmax(nested_logits, dim=-1), torch.argmax(rel_pos_logits, dim=-1)
            generated_ids = torch.cat([generated_ids, next_token_id], dim=1)
            for i in range(B):
                if not is_finished[i]:
                    new_pos_str = self._construct_next_pos_string(pos_strings_T[i][-1], pred_nested_level[i].item(), pred_rel_pos[i].item())
                    pos_strings_T[i].append(new_pos_str)
            is_finished |= (next_token_id.squeeze(-1) == self.eos_id)
            if is_finished.all(): break
        return generated_ids

    @torch.no_grad()
    def generate_beam_search(self, imgs, beam_size=5, max_gen_len=100):
        self.eval(); B = imgs.shape[0]; device = imgs.device
        img_mask = torch.zeros_like(imgs[:, 0, :, :], dtype=torch.bool)
        vis_feats, mem_pad_mask = self.encoder(imgs, img_mask)
        vis_feats = vis_feats.repeat_interleave(beam_size, dim=0)
        if mem_pad_mask is not None: mem_pad_mask = mem_pad_mask.repeat_interleave(beam_size, dim=0)
        effective_batch_size = B * beam_size
        generated_ids = torch.full((effective_batch_size, 1), self.sos_id, dtype=torch.long, device=device)
        pos_strings_T = [['M'] for _ in range(effective_batch_size)]
        log_scores = torch.zeros(effective_batch_size, device=device)
        is_finished = torch.zeros(effective_batch_size, dtype=torch.bool, device=device)
        for t in range(max_gen_len - 1):
            if is_finished.all(): break
            token_embeds = self.emb_tok(generated_ids)
            pos_ids_list = []
            for i in range(effective_batch_size):
                batch_pos_ids = []
                for p_str in pos_strings_T[i]:
                    ids = [self.pos_vocab.sos_id] + [self.pos_vocab.stoi.get(c, 0) for c in p_str] + [self.pos_vocab.stoi['<EOS>']]
                    padded_ids = ids[:MAX_IDENTIFIER_LEN] + [self.pos_vocab.pad_id] * (MAX_IDENTIFIER_LEN - len(ids))
                    batch_pos_ids.append(torch.tensor(padded_ids, device=device))
                pos_ids_list.append(torch.stack(batch_pos_ids))
            pos_matrix = torch.stack(pos_ids_list)
            pos_embeds = self.xi_proj(rearrange(self.pos_id_emb(pos_matrix), 'b l d e -> b l (d e)'))
            tgt = token_embeds + pos_embeds
            tgt_mask = torch.triu(torch.ones(tgt.size(1), tgt.size(1), device=device), 1).bool()
            dec_out = self.dec_norm(self._decode(tgt, vis_feats, generated_ids, tgt_mask, None, mem_pad_mask))
            last_step_out = dec_out[:, -1, :]
            token_logits, nested_logits, rel_pos_logits = self.head_tok(last_step_out), self.head_nested(last_step_out), self.head_rel_pos(last_step_out)
            log_probs = F.log_softmax(token_logits, dim=-1)
            if t > 0:
                log_probs[is_finished] = -float('inf')
                log_probs[is_finished, self.pad_id] = 0
            total_scores = log_probs + log_scores.unsqueeze(1)
            total_scores = total_scores.view(B, -1)
            top_scores, top_indices = torch.topk(total_scores, beam_size, dim=1)
            beam_indices = top_indices // len(self.vocab.itos)
            token_indices = top_indices % len(self.vocab.itos)
            batch_indices = torch.arange(B, device=device).view(-1, 1).repeat(1, beam_size)
            beam_indices_abs = beam_indices + (batch_indices * beam_size)
            generated_ids = generated_ids[beam_indices_abs.view(-1)]
            pos_strings_T = [pos_strings_T[i] for i in beam_indices_abs.view(-1).tolist()]
            generated_ids = torch.cat([generated_ids, token_indices.view(-1, 1)], dim=1)
            pred_nested_levels = torch.argmax(nested_logits[beam_indices_abs.view(-1)], dim=-1)
            pred_rel_poses = torch.argmax(rel_pos_logits[beam_indices_abs.view(-1)], dim=-1)
            for i in range(effective_batch_size):
                new_pos_str = self._construct_next_pos_string(pos_strings_T[i][-1], pred_nested_levels[i].item(), pred_rel_poses[i].item())
                pos_strings_T[i].append(new_pos_str)
            log_scores = top_scores.view(-1)
            is_finished = is_finished[beam_indices_abs.view(-1)] | (token_indices.view(-1) == self.eos_id)
        seq_lengths = (generated_ids != self.pad_id).sum(dim=1).float()
        seq_lengths = torch.max(seq_lengths, torch.ones_like(seq_lengths))
        normalized_scores = (log_scores / seq_lengths).view(B, beam_size)
        best_beam_indices = torch.argmax(normalized_scores, dim=1)
        final_indices = best_beam_indices + torch.arange(B, device=device) * beam_size
        return generated_ids[final_indices]
# =================================================================================
#                                  7. TRAINING & EVALUATION (TEMPLATE)
# =================================================================================





# ===== Loss Functions =====
import os
import math
import pickle
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm


def evaluate_model(model, loader, device, current_epoch=1, num_examples_to_print=5):
    """Оценивает модель с помощью генерации и сравнивает строки."""
    model.eval()
    corr_seq = 0
    total_seq = 0
    eos_id = model.eos_id

    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Epoch {current_epoch} Evaluating")
        for i, (imgs, _, _, _, _, _, latex_strings) in enumerate(pbar):
            imgs = imgs.to(device)
            #model.generate принимает только картинки
            pred_tokens = model.generate_beam_search(imgs)

            if i == 0: # Печатаем примеры только для первого батча
                print("\n" + "="*50)
                print(f"====== EXAMPLES FROM VALIDATION (EPOCH {current_epoch}) ======")
                num_to_print = min(num_examples_to_print, imgs.size(0))
                for j in range(num_to_print):
                    pred_ids = pred_tokens[j].cpu().tolist()
                    try:
                        eos_index = pred_ids.index(eos_id)
                        pred_ids = pred_ids[:eos_index]
                    except ValueError: pass
                    # Используем словарь из модели
                    pred_str = model.vocab.decode(pred_ids)
                    true_str = latex_strings[j]
                    # Сравниваем строки без пробелов
                    print(f"\n--- Example {j+1} ---")
                    print(f"  GROUND TRUTH: {true_str.replace(' ', '')}")
                    print(f"  PREDICTED   : {pred_str.replace(' ', '')}")
                print("="*50 + "\n")

            for j in range(imgs.size(0)):
                true_str_norm = latex_strings[j].replace(" ", "")
                pred_ids = pred_tokens[j].cpu().tolist()
                try:
                    eos_index = pred_ids.index(eos_id)
                    pred_ids = pred_ids[:eos_index]
                except ValueError: pass
                pred_str_norm = model.vocab.decode(pred_ids).replace(" ", "")

                if pred_str_norm == true_str_norm:
                    corr_seq += 1
                total_seq += 1

    seq_accuracy = corr_seq / max(total_seq, 1)
    return {'sequence_accuracy': seq_accuracy}

def create_optimizer_and_scheduler(model, total_steps, max_lr, weight_decay):
    """Создает AdamW и OneCycleLR scheduler."""
    no_decay = ['bias', 'norm']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=max_lr, betas=(0.9, 0.98), eps=1e-9)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps,
                                                  pct_start=0.1, anneal_strategy='cos')
    return optimizer, scheduler

def train(model, train_dataset, val_dataset, epochs, batch_size, device, save_path):
    """Основной цикл обучения модели."""
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=2, pin_memory=True)

    total_steps = len(train_loader) * epochs
    optimizer, scheduler = create_optimizer_and_scheduler(model, total_steps, MAX_LEARNING_RATE, WEIGHT_DECAY)

    start_epoch, best_seq_acc = load_checkpoint(model, optimizer, scheduler, save_path, device)
    for epoch in range(start_epoch, epochs + 1):
        model.train()
        epoch_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} Train Loss: N/A")

        for batch_idx, (imgs, masks, token_ids, pos_matrix, nested_gt, rel_pos_gt, _) in enumerate(pbar):
            imgs, masks, token_ids, pos_matrix, nested_gt, rel_pos_gt = \
                imgs.to(device), masks.to(device), token_ids.to(device), pos_matrix.to(device), nested_gt.to(device), rel_pos_gt.to(device)

            #  Прямой вызов модели
            logits = model(imgs, masks, pos_matrix, token_ids)
            targets = (token_ids, nested_gt, rel_pos_gt)
            # Считаем loss
            losses = model.compute_loss(logits, targets)
            loss = losses['total']

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Хорошая практика от градиентного взрыва
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()
            # Подсчёт среднего лосса
            pbar.set_description(f"Epoch {epoch} Train Loss: {epoch_loss / (batch_idx + 1):.4f} | LR: {scheduler.get_last_lr()[0]:.1e}")

        # Подсчет среднего лосса за всю эпоху
        avg_train_loss = epoch_loss / len(train_loader)

        # Валидация
        if epoch>80:
            metrics = evaluate_model(model, val_loader, device, current_epoch=epoch)
            seq_acc = metrics['sequence_accuracy']
            print(f"\nEpoch {epoch} Summary: Train Loss={avg_train_loss:.4f}, Val Seq Acc={seq_acc:.4f}\n")
            if seq_acc > best_seq_acc:
                best_seq_acc = seq_acc
                save_checkpoint(model, optimizer, scheduler, epoch, best_seq_acc, save_path)
        print(f"\nEpoch {epoch} Summary: Train Loss={avg_train_loss:.4f}\n")

    return model

In [ ]:
!rm best_model_onecyclelr_without_label_smoothing.pth

ERROR! Session/line number was not unique in database. History logging moved to new session 29


In [8]:
def save_checkpoint(model, optimizer, scheduler, epoch, best_acc, checkpoint_path):
    """Сохраняет состояние модели, оптимизатора и т.д. в файл."""
    print(f"\n==> Validation accuracy improved to {best_acc:.4f}. Saving checkpoint for epoch {epoch}...")
    state = {
        'epoch': epoch,
        'best_val_acc': best_acc,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
    }
    torch.save(state, checkpoint_path)
    print(f"Чекпоинт сохранен в {checkpoint_path}\n")

In [9]:
def load_checkpoint(model, optimizer, scheduler, checkpoint_path, device):
    """Загружает состояние модели, оптимизатора и т.д. из файла."""
    start_epoch = 1
    best_acc = 0.0
    if os.path.exists(checkpoint_path):
        print(f"==> Загрузка чекпоинта из {checkpoint_path}...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        if optimizer and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if scheduler and 'scheduler_state_dict' in checkpoint:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_acc = checkpoint.get('best_val_acc', 0.0)
        print(f"Чекпоинт загружен. Продолжаем с эпохи {start_epoch}, лучшая точность: {best_acc:.4f}")
    else:
        print("==> Чекпоинт не найден. Обучение с нуля.")
    return start_epoch, best_acc


['2019', '2014', '2016', 'train']


In [12]:
import os
import pickle
import torch
# Импортируем все необходимое

if __name__ == '__main__':
    # Параметры
    namphanvanhai_crohme_path = "/kaggle/input"
    base_dir = os.path.join(namphanvanhai_crohme_path,"CROHME","train")
    caption_file = "caption.txt"
    val_base_dir = os.path.join(namphanvanhai_crohme_path,"CROHME","2014")
    val_caption_file = "caption.txt"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    SAVE_PATH = "posformer_best_v4_lr_off.pth"
    VOCAB_PATH = "crohme_vocab.pkl"

    print("Загрузка данных для словаря...")
    all_latex = []
    with open(os.path.join(base_dir, caption_file), encoding='utf-8') as f:
        for line in f: all_latex.append(line.strip().split('\t', 1)[1])
    with open(os.path.join(val_base_dir, val_caption_file), encoding='utf-8') as f:
        for line in f: all_latex.append(line.strip().split('\t', 1)[1])

    print("Создание словарей...")
    vocab = Vocab(all_latex)
    pos_vocab = PosVocab()

    with open(VOCAB_PATH, "wb") as f:
        pickle.dump(vocab, f)
    print(f"Словарь сохранен в {VOCAB_PATH}, размер: {len(vocab.itos)} токенов.")

    print("Создание датасетов...")
    # Используем один и тот же `vocab` для обоих наборов
    train_dataset = CROHMEDataset(base_dir, caption_file, vocab, pos_vocab, is_train=True)
    val_dataset = CROHMEDataset(val_base_dir, val_caption_file, vocab, pos_vocab, is_train=False)
    print(f"Размер обучающего набора: {len(train_dataset)}, валидационного: {len(val_dataset)}")

    print("Инициализация модели...")
    model = PosFormer(main_vocab=vocab, pos_vocab=pos_vocab).to(device)

    # 5) Запуск обучения
    train(
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        device=device,
        save_path=SAVE_PATH
    )

Загрузка данных для словаря...
Создание словарей...
Словарь сохранен в crohme_vocab.pkl, размер: 114 токенов.
Создание датасетов...
Размер обучающего набора: 8834, валидационного: 986
Инициализация модели...
==> Загрузка чекпоинта из posformer_best_v4_lr_off.pth...
Чекпоинт загружен. Продолжаем с эпохи 59, лучшая точность: 0.1410


Epoch 59 Train Loss: 0.8887 | LR: 2.9e-04: 100%|██████████| 1105/1105 [04:42<00:00,  3.91it/s]



Epoch 59 Summary: Train Loss=0.8887



Epoch 60 Train Loss: 0.8900 | LR: 2.9e-04: 100%|██████████| 1105/1105 [04:43<00:00,  3.90it/s]



Epoch 60 Summary: Train Loss=0.8900



...


====== EXAMPLES FROM VALIDATION (EPOCH 103) ======

--- Example 1 ---
  GROUND TRUTH: 2p
  PREDICTED   : 2k=\sqrt{x\cos\theta}

--- Example 2 ---
  GROUND TRUTH: kN
  PREDICTED   : k_{1,-2n+13m}

--- Example 3 ---
  GROUND TRUTH: -7
  PREDICTED   : -\frac{\sqrt{x+yww}(z)}}{m}}

--- Example 4 ---
  GROUND TRUTH: 20
  PREDICTED   : 20,\sin\infty

--- Example 5 ---
  GROUND TRUTH: 12
  PREDICTED   : 1,\cos\alpha\sin\frac{2}{13}



Epoch 103 Evaluating: 100%|██████████| 124/124 [02:52<00:00,  1.39s/it]



Epoch 103 Summary: Train Loss=0.8491, Val Seq Acc=0.2028


Epoch 103 Summary: Train Loss=0.8491



Epoch 104 Train Loss: 0.8514 | LR: 2.5e-04:  25%|██▌       | 281/1105 [01:12<03:31,  3.89it/s]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
import pickle
from PIL import Image
from torchvision import transforms
import os
from einops import rearrange


@torch.no_grad()
def generate_beam_search(model, image_tensor, beam_size, max_len, device):
    """
    ИСПРАВЛЕННАЯ ВЕРСИЯ Beam Search для модели PosFormer.
    """
    model.eval()
    model.reset_iac_state()

    # 1. Энкодер вызывается один раз
    img_mask = torch.zeros_like(image_tensor[:, 0, :, :], dtype=torch.bool)
    vis_feats, mem_pad_mask = model.encoder(image_tensor.unsqueeze(0), img_mask.unsqueeze(0))
    B, Lk, D = vis_feats.shape
    # Расширяем для beam search
    vis_feats = vis_feats.expand(beam_size, Lk, D)
    if mem_pad_mask is not None:
        mem_pad_mask = mem_pad_mask.expand(beam_size, Lk)

    # 2. Инициализация лучей
    # Каждый луч: (последовательность, логарифмическая вероятность)
    k_beams = [(torch.full((1, 1), model.sos_id, dtype=torch.long, device=device), 0.0)]

    for _ in range(max_len - 1):
        all_candidates = []
        # Расширяем IAC состояние для каждого луча
        for seq, score in k_beams:
            if seq[0, -1] == model.eos_id:
                all_candidates.append((seq, score))
                continue

            # --- КОРРЕКТНЫЙ ВЫЗОВ ДЕКОДЕРА ---
            # Готовим TGT из уже сгенерированных токенов (не из Position Forest)
            tgt_embeds = model.emb_tok(seq) + model.dec_pos_enc[:, :seq.size(1), :]
            tgt_mask = torch.triu(torch.ones(seq.size(1), seq.size(1), device=device), 1).bool()

            # В инференсе с beam=1 мы бы передали (tgt_embeds, vis_feats[0], ...),
            # но т.к. vis_feats уже расширен, мы можем просто передать его как есть.
            # Мы декодируем по одному лучу за раз
            dec_out = model.dec_norm(model._decode(tgt_embeds, vis_feats[0:1], seq, tgt_mask, None, mem_pad_mask[0:1] if mem_pad_mask is not None else None))

            logits = model.head_tok(dec_out[:, -1, :])
            log_probs = F.log_softmax(logits, dim=-1)

            top_log_probs, top_ids = torch.topk(log_probs, beam_size, dim=-1)

            for i in range(beam_size):
                next_id = top_ids[0, i].unsqueeze(0).unsqueeze(0)
                new_seq = torch.cat([seq, next_id], dim=1)
                new_score = score + top_log_probs[0, i].item()
                all_candidates.append((new_seq, new_score))

        # Сортируем всех кандидатов и выбираем `beam_size` лучших
        ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
        k_beams = ordered[:beam_size]

        if all(b[0][0, -1] == model.eos_id for b in k_beams): break

    # Нормализуем на длину и возвращаем лучший
    best_beam = sorted(k_beams, key=lambda x: x[1] / len(x[0][0]), reverse=True)[0]
    return best_beam[0].squeeze(0)


def predict(model, image_path, vocab, beam_size, device):
    """Обертка для загрузки изображения и запуска предсказания."""
    # ИСПРАВЛЕНО: Трансформация должна соответствовать обучению (1 канал, нормализация 0.5)
    resizer = ResizeWithPadding(IMG_HEIGHT, IMG_WIDTH, padding_value=255) # Белый фон для паддинга
    img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

    try:
        img = Image.open(image_path).convert('L') # Конвертируем в grayscale
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден {image_path}")
        return ""

    img, _ = resizer(img)
    image_tensor = img_transform(img).to(device)

    predicted_ids = generate_beam_search(model, image_tensor, beam_size, MAX_SEQ_LEN, device)

    # Декодирование
    pred_list = predicted_ids.cpu().tolist()
    try:
        eos_index = pred_list.index(model.eos_id)
        pred_list = pred_list[:eos_index]
    except ValueError: pass

    return vocab.decode(pred_list)


if __name__ == '__main__':
    CHECKPOINT_PATH = "posformer_best.pth"
    VOCAB_PATH = "crohme_vocab.pkl"
    IMAGE_TO_PREDICT = "/path/to/your/image.png"
    BEAM_SIZE = 5

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("Загрузка словаря...")
    with open(VOCAB_PATH, 'rb') as f: vocab = pickle.load(f)
    pos_vocab = PosVocab()

    print("Инициализация модели...")
    # ИСПРАВЛЕНО: Используем правильный конструктор
    model = PosFormer(main_vocab=vocab, pos_vocab=pos_vocab).to(device)

    # Загружаем веса
    load_checkpoint(model, None, None, CHECKPOINT_PATH, device)

    if not os.path.exists(IMAGE_TO_PREDICT):
        print(f"ОШИБКА: Файл изображения не найден: {IMAGE_TO_PREDICT}")
    else:
        print("\nНачинаю распознавание...")
        predicted_latex = predict(model, IMAGE_TO_PREDICT, vocab, BEAM_SIZE, device)
        print("="*30)
        print(f"Распознанная формула: {predicted_latex}")
        print("="*30)

Загрузка словаря...
Инициализация модели...
Загрузка весов из /kaggle/working/best_model_onecyclelr_without_label_smoothing.pth...
Веса успешно загружены.

Начинаю распознавание...
Распознанная формула: f(x)=\sqrt{x^{2}}


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Efficient Beam search result: [1, 22, 24, 81, 85, 29, 84, 94, 28, 59, 5, 21, 86, 113, 86, 42, 36, 86, 45, 5, 48, 91, 10, 113, 48, 74, 10, 42, 36, 113, 38, 5, 74, 42, 16, 99, 73, 113, 5, 74, 5, 70, 98, 84, 98, 24, 20, 72, 5, 86, 73, 42, 66, 84, 45, 4, 83, 42, 39, 113, 21, 22, 79, 86, 89, 86, 45, 67, 102, 74, 42, 113, 99, 5, 68, 73, 113, 56, 86, 80, 18, 86, 40, 71, 40, 77, 73, 16, 72, 14, 72, 5, 9, 25, 56, 67, 56, 59, 58, 58]
